<a href="https://colab.research.google.com/github/cgangwar11/Coding-Practice/blob/master/practice_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://download.pytorch.org/tutorial/data.zip

--2019-03-26 16:44:50--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.35.115.56, 13.35.115.18, 13.35.115.116, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.35.115.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  17.0MB/s    in 0.2s    

2019-03-26 16:44:51 (17.0 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [6]:
!ls data/

eng-fra.txt  names


### Data Preprocessing


In [0]:
import glob
import unicodedata
import string




In [0]:
file_list = glob.glob('data/names/*.txt')

In [0]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [0]:
file = open(file_list[0],'r')

In [0]:
out = list(file.readlines())

In [0]:
def readlines(filename):
    out = open(filename).read().strip().split('\n')
    return [unicode_to_ascii(i) for i in out]



In [0]:
all_categories = []
names = {}
category_to_int = {}
for file in file_list:
    category = file.split('/')[-1].split('.txt')[0]
    category = category.strip().lower()
    names[category] = readlines(file)
    all_categories.append(category)

In [0]:
category_to_int = {j:i for i,j in enumerate(all_categories)}
all_letters_to_int = {j:i for i,j in enumerate(all_letters)}
def category_to_tensor(i):
    out=torch.zeros(1,len(all_categories))
    out[0,category_to_int[i]]=1
    return out

def word_to_tensor(word):
    out = []
    for i in word:
        out.append(all_letters_to_int[i])
    return torch.Tensor(out).view(1,len(out),-1)
        
        

In [77]:
word_to_tensor('Saleem').shape

torch.Size([1, 6, 1])

In [0]:
k[0,5]=1

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [44]:
print(all_letters)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'


In [0]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

In [0]:
import torch.nn as nn
import torch.functional as F
import torch

In [0]:
class RNN(nn.Module):
    def __init__(self,num_embedding,embedding_dim,hidden_size):
        super(RNN,self).__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embedding,embedding_dim=embedding_dim)
        self.rnn = nn.RNN(input_size = embedding_dim,hidden_size=hidden_size)
        
    def forward(self,input):
        out = self.embedding(input)
       
        output,_ = self.rnn(out)
        return output[:,-1,:]
        

In [0]:
model = RNN(27,5,5)

In [0]:
model(torch.LongTensor([[5,6,7,8],[10,1,2,4]])).shape

torch.Size([2, 5])